In [2]:
import pandas as pd
from tqdm import tqdm
from transformers import pipeline

# Set pandas to display all columns
pd.set_option('display.max_columns', None)

In [91]:
df = pd.read_excel('llama_merged.xlsx')
df.head(2)

,id,author_id,in_reply_to_user_id,quoted_user_id,retweeted_user_id,text,public_metrics.like_count,public_metrics.quote_count,public_metrics.reply_count,public_metrics.retweet_count,entities.hashtags,entities.mentions,author.username,author.name,author.description,author.location,author.public_metrics.followers_count,author.public_metrics.following_count,author.public_metrics.tweet_count,url,link_text,party,name,Occupation(s),Education,Residence[5],gpt_sentiment,age,gender,bert_sentiment,no_polit,no_pol_gpt_label,no_polit_bert_class,llama_using,llama_arguing,llama_name,llama_gender,llama_edu,llama_age
0,1174499599181958912,262192574,NaN,NaN,NaN,We should never forget the sacrifices made by ...,22,0,0,2,NaN,NaN,cindyhydesmith,Cindy Hyde-Smith,Lifelong conservative proudly fighting for the...,"Mississippi, USA",20222,199,2232,https://www.wdam.com/2019/09/18/senator-cindy-...,MISSISSIPPI (WLOX) - U.S. Senator Cindy Hyde-S...,Republican,Cindy Hyde-Smith,Lobbyist Farmer,Copiah-Lincoln Community College (AA) Universi...,Brookhaven,positive,65,female,positive,"""Kudos to @SenHydeSmith for introducing Gold S...",positive,positive,NaN,NaN,NaN,"""Proud to see my colleague @SenHydeSmith leadi...",NaN,NaN
1,1320738770543628032,15808765,NaN,NaN,NaN,It's no secret what Donald Trump and Senate Re...,1494,67,264,323,NaN,NaN,CoryBooker,Cory Booker,U.S. Senator from New Jersey. Most tweets (and...,"Newark, NJ",4835171,85102,67749,https://thehill.com/homenews/administration/52...,President Trump says in an interview with “60 ...,Democrat,Cory Booker,Lawyer,"Stanford University (BA, MA) Queen's College, ...",Newark,negative,55,male,negative,# #60Minutes #TrumpCareCant believe @realDonal...,negative,negative,Biden has vowed to protect the law and even e...,NaN,Biden also emphasized his commitment to build...,Biden has vowed to build on ObamaCare and exp...,Biden has vowed to protect and strengthen the...,Biden has repeatedly made the case that Trump...


In [49]:
# set pipeline for sentiment labelling
sentiment_pipeline = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest", tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [127]:
df['llama_age'] = df['llama_age'].astype(str)  #llama_age

# Enable tqdm for pandas
tqdm.pandas()

# apply sentiment classification and create 'BERT_class' column for Twitter DataFrame
df['bert_class1'] = df['llama_age'].progress_apply(lambda x: sentiment_pipeline(x)[0]['label'])

100%|█████████████████████████████████████| 10000/10000 [08:13<00:00, 20.27it/s]


In [128]:
# apply sentiment classification and create 'BERT_class' column for Twitter DataFrame
df['bert_class2'] = df['llama_age'].progress_apply(lambda x: sentiment_pipeline(x)[0]['label'])

100%|█████████████████████████████████████| 10000/10000 [08:33<00:00, 19.49it/s]


In [129]:
# apply sentiment classification and create 'BERT_class' column for Twitter DataFrame
df['bert_class3'] = df['llama_age'].progress_apply(lambda x: sentiment_pipeline(x)[0]['label'])

100%|█████████████████████████████████████| 10000/10000 [08:53<00:00, 18.73it/s]


In [115]:
def determine_final_class(row):
    classes = [row['bert_class1'], row['bert_class2'], row['bert_class3']]
    # Count occurrences of each class
    class_counts = pd.Series(classes).value_counts()
    
    # If the highest count is greater than 1, return that class (majority vote)
    if class_counts.iloc[0] > 1:
        return class_counts.idxmax()
    else:
        # If there's no majority, you can decide on a default or handle differently
        return 'no consensus'

In [130]:
df['llama_age_bert_class'] = df.apply(determine_final_class, axis=1)
df['llama_age_bert_class'].unique()

array(['neutral', 'negative', 'positive'], dtype=object)

In [131]:
df = df.drop(columns=['bert_class1', 'bert_class2', 'bert_class3'])

In [132]:
df.columns

Index(['id', 'author_id', 'in_reply_to_user_id', 'quoted_user_id',
       'retweeted_user_id', 'text', 'public_metrics.like_count',
       'public_metrics.quote_count', 'public_metrics.reply_count',
       'public_metrics.retweet_count', 'entities.hashtags',
       'entities.mentions', 'author.username', 'author.name',
       'author.description', 'author.location',
       'author.public_metrics.followers_count',
       'author.public_metrics.following_count',
       'author.public_metrics.tweet_count', 'url', 'link_text', 'party',
       'name', 'Occupation(s)', 'Education', 'Residence[5]', 'gpt_sentiment',
       'age', 'gender', 'bert_sentiment', 'no_polit', 'no_pol_gpt_label',
       'no_polit_bert_class', 'llama_using', 'llama_arguing', 'llama_name',
       'llama_gender', 'llama_edu', 'llama_age', 'llama_using_bert_class',
       'llama_arguing_bert_class', 'llama_name_bert_class',
       'llama_gender_bert_class', 'llama_edu_bert_class',
       'llama_age_bert_class'],
      dty

In [133]:
# save
df.to_csv('llama_full_merged.csv', index=False)
df.to_excel('llama_full_merged.xlsx', index=False)